In [1]:
import pandas as pd
import os

In [30]:
def list_files(files):
    """Accepts a list and returns a dictionary of each item in list with a numbered index (key) starting at 1"""

    dir_list = {0:'Exit'}
    file_count = 0
    for file in files:
        file_count +=1
        dir_list[file_count] = file  # assign a number to each item in the list starting with 1 
    return dir_list  # returns the dictionary of items in the list

In [31]:
def print_dict(dict_items):
    """Prints the key:value pairing in a dictionary"""

    for key, value in dict_items.items():
        print(key, ":", value)

In [32]:
def check_dict(dict_to_check, choice):
    """Checks if the input choice is a key in the provided dictionary
        and returns 'valid' if true, 'invalid' if false"""

    if choice in dict_to_check:
        key_exists = "valid"
    else:
        key_exists = "invalid"

    return key_exists

In [33]:
def error_message(invalid_choice):
    print(f"Your input of '{invalid_choice}' is not a valid choice.")

In [38]:
dirs = sorted(os.listdir('data'))
dir_list = list_files(dirs)  # returned dict of files in directory

print_dict(dir_list)
user_choice = int(input('Enter the number next to the file to open. (0 to exit): '))
validate_choice = check_dict(dir_list, user_choice) # check to see if user input is a key in the dictionary

while validate_choice == "invalid":
    error_message(user_choice)
    print_dict(dir_list)
    user_choice = int(input('Enter the number next to the file to open. (0 to exit): '))
    validate_choice = check_dict(dir_list, user_choice) # check to see if user input is a key in the dictionary
    if user_choice == 0:
        break
    
# while user_choice != 0:


0 : Exit
1 : ingred.csv
2 : p2.csv
Your input of '5' is not a valid choice.
0 : Exit
1 : ingred.csv
2 : p2.csv
Your input of '6' is not a valid choice.
0 : Exit
1 : ingred.csv
2 : p2.csv
Your input of '7' is not a valid choice.
0 : Exit
1 : ingred.csv
2 : p2.csv
Your input of '5' is not a valid choice.
0 : Exit
1 : ingred.csv
2 : p2.csv
